In [ ]:
import pandas as pd
data = pd.read_excel('/content/training.xlsx')

In [ ]:
df=data

In [ ]:
df

,Sentence,Category,Solution
0,A light switch is not working and the lights w...,Electricity,Repair
1,The air conditioner is making strange noises a...,Air Conditioner,Repair
2,The air conditioner is blowing warm air instea...,Air Conditioner,Maintenance
3,Broken faucet causing continuous dripping and ...,Plumbing,Repair
4,The bathroom sink is clogged and water is not ...,Plumbing,Maintenance
...,...,...,...
95,The computer's graphics card is not displaying...,Computer,Repair
96,The washing machine is not draining water prop...,Washer,Repair
97,The refrigerator door gasket is worn and needs...,Refrigerator,Repair
98,The outdoor lighting system needs to have new ...,Electricity,Installation


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Prepare the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Sentence'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Sentence'])
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Prepare the labels
category_labels = pd.get_dummies(df['Category']).values
solution_labels = pd.get_dummies(df['Solution']).values


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate

# Define the input
input_ = Input(shape=(max_sequence_length,))

# Embedding layer
embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length)(input_)

# LSTM layer
lstm = LSTM(128)(embedding)

# Dense layers for category and action predictions
category_output = Dense(len(df['Category'].unique()), activation='softmax', name='category_output')(lstm)
solution_output = Dense(len(df['Solution'].unique()), activation='softmax', name='solution_output')(lstm)

# Combine the model
model = Model(inputs=input_, outputs=[category_output, solution_output])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 21)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 21, 128)              50688     ['input_2[0][0]']             
                                                                                                  
 lstm_1 (LSTM)               (None, 128)                  131584    ['embedding_1[0][0]']         
                                                                                                  
 category_output (Dense)     (None, 7)                    903       ['lstm_1[0][0]']              
                                                                                            

In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train_category, y_val_category, y_train_solution, y_val_solution = train_test_split(
    padded_sequences, category_labels, solution_labels, test_size=0.2, random_state=42
)

# Train the model
history = model.fit(
    X_train,
    {'category_output': y_train_category, 'solution_output': y_train_solution},
    validation_data=(X_val, {'category_output': y_val_category, 'solution_output': y_val_solution}),
    epochs=100,
    batch_size=32
)


Epoch 1/100
3/3 [==============================] - 5s 444ms/step - loss: 3.3055 - category_output_loss: 1.9451 - solution_output_loss: 1.3604 - category_output_accuracy: 0.2000 - solution_output_accuracy: 0.4875 - val_loss: 3.2819 - val_category_output_loss: 1.9340 - val_solution_output_loss: 1.3480 - val_category_output_accuracy: 0.1500 - val_solution_output_accuracy: 0.4000
Epoch 2/100
3/3 [==============================] - 0s 123ms/step - loss: 3.2273 - category_output_loss: 1.9333 - solution_output_loss: 1.2940 - category_output_accuracy: 0.2125 - solution_output_accuracy: 0.5250 - val_loss: 3.2292 - val_category_output_loss: 1.9162 - val_solution_output_loss: 1.3130 - val_category_output_accuracy: 0.1500 - val_solution_output_accuracy: 0.4000
Epoch 3/100
3/3 [==============================] - 0s 120ms/step - loss: 3.1371 - category_output_loss: 1.9138 - solution_output_loss: 1.2233 - category_output_accuracy: 0.2125 - solution_output_accuracy: 0.5250 - val_loss: 3.1815 - val_categ

Generate Responses

In [ ]:
import numpy as np

def generate_response(description):
    # Preprocess the input description
    description_sequence = tokenizer.texts_to_sequences([description])
    description_padded = pad_sequences(description_sequence, maxlen=max_sequence_length, padding='post')

    # Predict using the model
    category_prob, solution_prob = model.predict(description_padded)

    # Get the predicted category and solution
    predicted_category = np.argmax(category_prob)
    predicted_solution = np.argmax(solution_prob)

    # Map indices back to category and solution labels
    category_label = df['Category'].unique()[predicted_category]
    solution_label = df['Solution'].unique()[predicted_solution]

    # Prepare the response
    response = f"For the issue, our recommendation is to check '{category_label}' and consider '{solution_label}'."

    return response

# Test the function
description = input("Please describe your issue: ")
response = generate_response(description)
print(response)

Please describe your issue: The computer's graphics card is not displaying properly and may need to be replaced or have drivers updated.
1/1 [==============================] - 0s 28ms/step
For the issue, our recommendation is to check 'Air Conditioner' and consider 'Repair '.


Save Model

In [ ]:
model.save("issue_classification_model.h5")

# Verify the saved model
loaded_model = tf.keras.models.load_model("issue_classification_model.h5")
print("Model loaded successfully and is ready for use.")


Model loaded successfully and is ready for use.
